# Data Cleaning

In [7]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

#### Load Dataset

In [8]:
file_name = '../data/insuranceFraud.csv'
data = pd.read_csv(file_name)

In [9]:
data.head()

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported
0,328,48,521585,10/17/2014,OH,250/500,1000,1406.91,0,466132,...,2,YES,71610,6510,13020,52080,Saab,92x,2004,Y
1,228,42,342868,6/27/2006,IN,250/500,2000,1197.22,5000000,468176,...,0,?,5070,780,780,3510,Mercedes,E400,2007,Y
2,134,29,687698,9/6/2000,OH,100/300,2000,1413.14,5000000,430632,...,3,NO,34650,7700,3850,23100,Dodge,RAM,2007,N
3,256,41,227811,5/25/1990,IL,250/500,2000,1415.74,6000000,608117,...,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y
4,228,44,367455,6/6/2014,IL,500/1000,1000,1583.91,6000000,610706,...,1,NO,6500,1300,650,4550,Accura,RSX,2009,N


#### Handle Missing Values

In [10]:
# Replacing all the "?" values with NaN to make them easier to handle
data = data.replace('?', np.nan)

In [11]:
# checking for missing values
data.isna().sum()

months_as_customer               0
age                              0
policy_number                    0
policy_bind_date                 0
policy_state                     0
policy_csl                       0
policy_deductable                0
policy_annual_premium            0
umbrella_limit                   0
insured_zip                      0
insured_sex                      0
insured_education_level          0
insured_occupation               0
insured_hobbies                  0
insured_relationship             0
capital-gains                    0
capital-loss                     0
incident_date                    0
incident_type                    0
collision_type                 178
incident_severity                0
authorities_contacted           91
incident_state                   0
incident_city                    0
incident_location                0
incident_hour_of_the_day         0
number_of_vehicles_involved      0
property_damage                360
bodily_injuries     

In [12]:


# Creating an imputer instance to fill missing values with the most frequent value in each column
imputer = SimpleImputer(strategy='most_frequent')

# Imputing missing values for specific categorical columns
data['collision_type'] = imputer.fit_transform(data[['collision_type']]).ravel()
data['property_damage'] = imputer.fit_transform(data[['property_damage']]).ravel()
data['police_report_available'] = imputer.fit_transform(data[['police_report_available']]).ravel()
data['authorities_contacted'] = imputer.fit_transform(data[['authorities_contacted']]).ravel()

In [13]:
# Check for missing values again
data.isna().sum()

months_as_customer             0
age                            0
policy_number                  0
policy_bind_date               0
policy_state                   0
policy_csl                     0
policy_deductable              0
policy_annual_premium          0
umbrella_limit                 0
insured_zip                    0
insured_sex                    0
insured_education_level        0
insured_occupation             0
insured_hobbies                0
insured_relationship           0
capital-gains                  0
capital-loss                   0
incident_date                  0
incident_type                  0
collision_type                 0
incident_severity              0
authorities_contacted          0
incident_state                 0
incident_city                  0
incident_location              0
incident_hour_of_the_day       0
number_of_vehicles_involved    0
property_damage                0
bodily_injuries                0
witnesses                      0
police_rep

#### Remove Duplicates

In [14]:
# Checking for duplicates
data.duplicated().sum()

# There are no duplicates in the dataset

0

### Remove Unwanted Features

In [15]:
# list of columns not necessary for pfrediction
cols_to_drop=['policy_number','policy_bind_date','policy_state','insured_zip','incident_location','incident_date','incident_state','incident_city','insured_hobbies','auto_make','auto_model','auto_year']
data = data.drop(cols_to_drop, axis=1)

#### Handle Outliers

In [16]:
num_df = data.select_dtypes(include=['int64', 'float64'])

In [17]:

# Checking for potential outliers using IQR, without removing them yet
outliers_summary = {}

for col in num_df:
    Q1 = num_df[col].quantile(0.25)
    Q3 = num_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers_count = num_df[(num_df[col] < lower_bound) | (num_df[col] > upper_bound)].shape[0]
    
    # Storing summary information
    outliers_summary[col] = {
        "Lower Bound": lower_bound,
        "Upper Bound": upper_bound,
        "Outliers Count": outliers_count
    }

# Displaying the outliers summary
pd.DataFrame(outliers_summary).T

,Lower Bound,Upper Bound,Outliers Count
months_as_customer,-125.00000,517.00000,0.0
age,14.00000,62.00000,4.0
policy_deductable,-1750.00000,4250.00000,0.0
policy_annual_premium,600.47625,1904.82625,9.0
umbrella_limit,0.00000,0.00000,202.0
capital-gains,-76537.50000,127562.50000,0.0
capital-loss,-128750.00000,77250.00000,0.0
incident_hour_of_the_day,-10.50000,33.50000,0.0
number_of_vehicles_involved,-2.00000,6.00000,0.0
bodily_injuries,-3.00000,5.00000,0.0


In [18]:
# Cap outliers based on IQR bounds
for col in ['umbrella_limit', 'property_claim', 'vehicle_claim']:
    Q1 = num_df[col].quantile(0.25)
    Q3 = num_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Apply capping
    num_df[col] = np.where(num_df[col] < lower_bound, lower_bound, num_df[col])
    num_df[col] = np.where(num_df[col] > upper_bound, upper_bound, num_df[col])

print("Outliers have been capped to IQR bounds.")

Outliers have been capped to IQR bounds.


In [19]:
# Checking for outliers again after capping
outliers_summary = {}

for col in ['umbrella_limit', 'property_claim', 'vehicle_claim']:
    Q1 = num_df[col].quantile(0.25)
    Q3 = num_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers_count = num_df[(num_df[col] < lower_bound) | (num_df[col] > upper_bound)].shape[0]
    
    # Storing summary information
    outliers_summary[col] = {
        "Lower Bound": lower_bound,
        "Upper Bound": upper_bound,
        "Outliers Count": outliers_count
    }

# Displaying the outliers summary after capping
pd.DataFrame(outliers_summary).T

,Lower Bound,Upper Bound,Outliers Count
umbrella_limit,0.0,0.0,0.0
property_claim,-5215.0,20545.0,0.0
vehicle_claim,-502.5,81617.5,0.0


#### Normalize Data

In [20]:
scaler = MinMaxScaler()
scaled_num_df = scaler.fit_transform(num_df)

num_df = pd.DataFrame(scaled_num_df, columns=num_df.columns)

print("Feature scaling (Min-Max) applied to numerical columns.")

Feature scaling (Min-Max) applied to numerical columns.


In [21]:
# Check min and max after scaling
num_df.describe().loc[['min', 'max']]

,months_as_customer,age,policy_deductable,policy_annual_premium,umbrella_limit,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,injury_claim,property_claim,vehicle_claim
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


#### Encode Categorical Variables

In [22]:
cat_df = data.select_dtypes(include=['object']).copy()

In [23]:
# custom mapping for encoding
cat_df['policy_csl'] = cat_df['policy_csl'].map({'100/300' : 1, '250/500' : 2.5 ,'500/1000':5})
cat_df['insured_education_level'] = cat_df['insured_education_level'].map({'JD' : 1, 'High School' : 2,'College':3,'Masters':4,'Associate':5,'MD':6,'PhD':7})
cat_df['incident_severity'] = cat_df['incident_severity'].map({'Trivial Damage' : 1, 'Minor Damage' : 2,'Major Damage':3,'Total Loss':4})
cat_df['insured_sex'] = cat_df['insured_sex'].map({'FEMALE' : 0, 'MALE' : 1})
cat_df['property_damage'] = cat_df['property_damage'].map({'NO' : 0, 'YES' : 1})
cat_df['police_report_available'] = cat_df['police_report_available'].map({'NO' : 0, 'YES' : 1})
cat_df['fraud_reported'] = cat_df['fraud_reported'].map({'N' : 0, 'Y' : 1})

In [24]:
# auto encoding of categorical variables
for col in cat_df.drop(columns=['policy_csl','insured_education_level','incident_severity','insured_sex','property_damage','police_report_available','fraud_reported']).columns:
    cat_df= pd.get_dummies(cat_df, columns=[col], prefix = [col], drop_first=True)

In [25]:
# data after encoding
cat_df.head()

,policy_csl,insured_sex,insured_education_level,incident_severity,property_damage,police_report_available,fraud_reported,insured_occupation_armed-forces,insured_occupation_craft-repair,insured_occupation_exec-managerial,...,insured_relationship_unmarried,insured_relationship_wife,incident_type_Parked Car,incident_type_Single Vehicle Collision,incident_type_Vehicle Theft,collision_type_Rear Collision,collision_type_Side Collision,authorities_contacted_Fire,authorities_contacted_Other,authorities_contacted_Police
0,2.5,1,6,3,1,1,1,False,True,False,...,False,False,False,True,False,False,True,False,False,True
1,2.5,1,6,2,0,0,1,False,False,False,...,False,False,False,False,True,True,False,False,False,True
2,1.0,0,7,2,0,0,0,False,False,False,...,False,False,False,False,False,True,False,False,False,True
3,2.5,0,7,3,0,0,1,True,False,False,...,True,False,False,True,False,False,False,False,False,True
4,5.0,1,5,2,0,0,0,False,False,False,...,True,False,False,False,True,True,False,False,False,True


#### Save Cleaned Data

In [26]:
# combining the Numerical and categorical dataframes to get the final dataset
final_df=pd.concat([num_df,cat_df], axis=1)

In [27]:
final_df.head()

,months_as_customer,age,policy_deductable,policy_annual_premium,umbrella_limit,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,...,insured_relationship_unmarried,insured_relationship_wife,incident_type_Parked Car,incident_type_Single Vehicle Collision,incident_type_Vehicle Theft,collision_type_Rear Collision,collision_type_Side Collision,authorities_contacted_Fire,authorities_contacted_Other,authorities_contacted_Police
0,0.684760,0.644444,0.333333,0.603112,0.0,0.530348,1.000000,0.217391,0.000000,0.5,...,False,False,False,True,False,False,True,False,False,True
1,0.475992,0.511111,1.000000,0.473214,0.0,0.000000,1.000000,0.347826,0.000000,0.0,...,False,False,False,False,True,True,False,False,False,True
2,0.279749,0.222222,1.000000,0.606972,0.0,0.349254,1.000000,0.304348,0.666667,1.0,...,False,False,False,False,False,True,False,False,False,True
3,0.534447,0.488889,1.000000,0.608582,0.0,0.486567,0.438344,0.217391,0.000000,0.5,...,True,False,False,True,False,False,False,False,False,True
4,0.475992,0.555556,0.333333,0.712760,0.0,0.656716,0.585959,0.869565,0.000000,0.0,...,True,False,False,False,True,True,False,False,False,True


In [28]:
# combining the Numerical and categorical dataframes to get the final dataset
final_df=pd.concat([num_df,cat_df], axis=1)

In [29]:
# Exporting the final cleaned and preprocessed data to a CSV file
final_df.to_csv('../data/cleaned_auto_insurance_claims.csv', index=False)
print("Data exported to '../data/cleaned_auto_insurance_claims.csv' successfully!")

Data exported to '../data/cleaned_auto_insurance_claims.csv' successfully!


#### Summary

**Summary: Ready for Export and Next Steps**

Alright, our data is now fully prepped! We've taken care of:

- **Handling missing values**: Replaced missing values to ensure consistency.
- **Encoding categorical variables**: Converted categorical data to numerical format so models can work with it.
- **Outlier treatment**: Capped extreme values to keep our data balanced.
- **Feature scaling**: Standardized the numerical columns to a [0, 1] range for smoother model performance.

Now we’re ready to export this cleaned and processed DataFrame to a CSV file. This exported file will be our finalized dataset, and here’s what we can do with it next:

1. **Training and Testing**: Use this file to split the data into training and test sets for building and evaluating our machine learning models.
2. **Deployment**: Since the data is clean, standardized, and model-ready, we can also use this same file in deployment for real-world predictions.